# Advanced Polars Analytics with pyhdb-rs

This notebook demonstrates how to leverage the **zero-copy Arrow integration**
between SAP HANA and Polars for high-performance analytics.

## Why Zero-Copy Matters

Traditional workflow:
```
HANA → Network → Python objects → pandas/numpy → Analysis
       ↑ slow        ↑ GC pressure      ↑ memory copy
```

pyhdb-rs workflow:
```
HANA → Network → Arrow buffers → Polars
                 ↑ zero-copy, no Python objects!
```

In [ ]:
import os
from pyhdb_rs import connect
import polars as pl

HANA_URL = os.environ.get("HANA_TEST_URI")

# Polars configuration for large datasets
pl.Config.set_tbl_rows(20)
pl.Config.set_fmt_str_lengths(50)

## LazyFrame for Deferred Execution

Polars LazyFrames allow query optimization before execution.
Combined with HANA's query pushdown, you get optimal performance.

In [ ]:
def load_sales_data() -> pl.LazyFrame:
    """Load sales data as LazyFrame for deferred processing."""
    with connect(HANA_URL) as conn:
        with conn.cursor() as cursor:
            # Push filtering to HANA - only transfer what we need
            df = cursor.execute_polars("""
                SELECT 
                    SALE_ID,
                    SALE_DATE,
                    CUSTOMER_ID,
                    PRODUCT_ID,
                    QUANTITY,
                    UNIT_PRICE,
                    DISCOUNT,
                    REGION
                FROM SALES
                WHERE SALE_DATE >= '2024-01-01'
            """)
            return df.lazy()

# Create LazyFrame - no computation yet!
sales_lf = load_sales_data()

In [ ]:
# Define transformations (still lazy - no execution)
result = (
    sales_lf
    .with_columns([
        # Calculate total amount
        (pl.col("QUANTITY") * pl.col("UNIT_PRICE") * (1 - pl.col("DISCOUNT"))).alias("TOTAL"),
        # Extract month
        pl.col("SALE_DATE").dt.month().alias("MONTH"),
    ])
    .filter(pl.col("TOTAL") > 100)  # Filter calculated column
    .group_by(["REGION", "MONTH"])
    .agg([
        pl.col("TOTAL").sum().alias("REVENUE"),
        pl.col("SALE_ID").count().alias("ORDER_COUNT"),
        pl.col("TOTAL").mean().alias("AVG_ORDER_VALUE"),
    ])
    .sort(["REGION", "MONTH"])
)

# Now execute and collect results
monthly_revenue = result.collect()
print(monthly_revenue)

## Window Functions

Polars provides powerful window functions for running totals, rankings, and more.

In [ ]:
with connect(HANA_URL) as conn:
    with conn.cursor() as cursor:
        df = cursor.execute_polars("""
            SELECT 
                EMPLOYEE_ID,
                DEPARTMENT,
                SALE_DATE,
                AMOUNT
            FROM EMPLOYEE_SALES
            WHERE SALE_DATE >= '2024-01-01'
        """)

# Window functions in Polars
result = df.with_columns([
    # Running total per employee
    pl.col("AMOUNT")
        .cum_sum()
        .over("EMPLOYEE_ID")
        .alias("RUNNING_TOTAL"),
    
    # Rank within department
    pl.col("AMOUNT")
        .rank(descending=True)
        .over("DEPARTMENT")
        .alias("DEPT_RANK"),
    
    # Percentage of department total
    (pl.col("AMOUNT") / pl.col("AMOUNT").sum().over("DEPARTMENT") * 100)
        .round(2)
        .alias("DEPT_PERCENTAGE"),
    
    # Moving average (7-day)
    pl.col("AMOUNT")
        .rolling_mean(window_size=7)
        .over("EMPLOYEE_ID")
        .alias("MA_7D"),
])

print(result.head(10))

## Efficient Joins

Load dimension tables once, join in Polars for repeated analysis.

In [ ]:
with connect(HANA_URL) as conn:
    with conn.cursor() as cursor:
        # Load fact table (large)
        orders = cursor.execute_polars("""
            SELECT ORDER_ID, CUSTOMER_ID, PRODUCT_ID, QUANTITY, ORDER_DATE
            FROM ORDERS 
            WHERE ORDER_DATE >= '2024-01-01'
        """)
        
        # Load dimension tables (small, can cache)
        customers = cursor.execute_polars("""
            SELECT CUSTOMER_ID, CUSTOMER_NAME, SEGMENT, COUNTRY
            FROM CUSTOMERS
        """)
        
        products = cursor.execute_polars("""
            SELECT PRODUCT_ID, PRODUCT_NAME, CATEGORY, UNIT_PRICE
            FROM PRODUCTS
        """)

print(f"Orders: {len(orders):,} rows")
print(f"Customers: {len(customers):,} rows")
print(f"Products: {len(products):,} rows")

In [ ]:
# Join and analyze in Polars (very fast!)
enriched = (
    orders
    .join(customers, on="CUSTOMER_ID", how="left")
    .join(products, on="PRODUCT_ID", how="left")
    .with_columns(
        (pl.col("QUANTITY") * pl.col("UNIT_PRICE")).alias("TOTAL")
    )
)

# Segment analysis
segment_analysis = (
    enriched
    .group_by(["SEGMENT", "CATEGORY"])
    .agg([
        pl.col("TOTAL").sum().alias("REVENUE"),
        pl.col("ORDER_ID").n_unique().alias("ORDERS"),
        pl.col("CUSTOMER_ID").n_unique().alias("CUSTOMERS"),
    ])
    .with_columns(
        (pl.col("REVENUE") / pl.col("ORDERS")).round(2).alias("AVG_ORDER_VALUE")
    )
    .sort("REVENUE", descending=True)
)

print(segment_analysis)

## Pivot Tables

Create pivot tables for cross-tabulation analysis.

In [ ]:
# Monthly revenue by region (pivot)
monthly_pivot = (
    enriched
    .with_columns(pl.col("ORDER_DATE").dt.strftime("%Y-%m").alias("MONTH"))
    .group_by(["COUNTRY", "MONTH"])
    .agg(pl.col("TOTAL").sum().alias("REVENUE"))
    .pivot(
        on="MONTH",
        index="COUNTRY",
        values="REVENUE",
    )
    .fill_null(0)
)

print(monthly_pivot)

## Time Series Analysis

Polars has excellent support for time series operations.

In [ ]:
with connect(HANA_URL) as conn:
    with conn.cursor() as cursor:
        # Load time series data
        metrics = cursor.execute_polars("""
            SELECT 
                TIMESTAMP,
                SENSOR_ID,
                TEMPERATURE,
                HUMIDITY,
                PRESSURE
            FROM IOT_METRICS
            WHERE TIMESTAMP >= ADD_DAYS(CURRENT_TIMESTAMP, -30)
            ORDER BY TIMESTAMP
        """)

# Resample to hourly aggregates
hourly = (
    metrics
    .sort("TIMESTAMP")
    .group_by_dynamic(
        "TIMESTAMP",
        every="1h",
        group_by="SENSOR_ID",
    )
    .agg([
        pl.col("TEMPERATURE").mean().alias("AVG_TEMP"),
        pl.col("TEMPERATURE").min().alias("MIN_TEMP"),
        pl.col("TEMPERATURE").max().alias("MAX_TEMP"),
        pl.col("HUMIDITY").mean().alias("AVG_HUMIDITY"),
        pl.len().alias("READINGS"),
    ])
)

print(f"Resampled to {len(hourly):,} hourly records")
print(hourly.head())

In [ ]:
# Detect anomalies using rolling statistics
anomalies = (
    hourly
    .sort(["SENSOR_ID", "TIMESTAMP"])
    .with_columns([
        # Rolling mean and std
        pl.col("AVG_TEMP")
            .rolling_mean(window_size=24)
            .over("SENSOR_ID")
            .alias("ROLLING_MEAN"),
        pl.col("AVG_TEMP")
            .rolling_std(window_size=24)
            .over("SENSOR_ID")
            .alias("ROLLING_STD"),
    ])
    .with_columns(
        # Z-score for anomaly detection
        ((pl.col("AVG_TEMP") - pl.col("ROLLING_MEAN")) / pl.col("ROLLING_STD"))
            .abs()
            .alias("Z_SCORE")
    )
    .filter(pl.col("Z_SCORE") > 3)  # More than 3 standard deviations
)

print(f"Detected {len(anomalies)} anomalies")
print(anomalies)

## Export Results

Polars supports various output formats.

In [ ]:
# Export to Parquet (columnar, compressed)
segment_analysis.write_parquet("segment_analysis.parquet")

# Export to CSV
segment_analysis.write_csv("segment_analysis.csv")

# Export to JSON
segment_analysis.write_json("segment_analysis.json")

# Convert to pandas for visualization libraries
pandas_df = segment_analysis.to_pandas()

print("Exported to parquet, csv, and json")

## Memory Efficiency

Check memory usage of your DataFrames.

In [ ]:
def memory_usage_mb(df: pl.DataFrame) -> float:
    """Calculate DataFrame memory usage in MB."""
    return df.estimated_size() / 1024 / 1024

print(f"Orders: {memory_usage_mb(orders):.2f} MB")
print(f"Enriched: {memory_usage_mb(enriched):.2f} MB")
print(f"Segment Analysis: {memory_usage_mb(segment_analysis):.2f} MB")